# Setup

In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Import Data
#from google.colab import drive
#drive.mount('/content/drive')
#import os
#os.listdir('/content/drive/MyDrive/Uni2025/IFN850 Machine Learning/Assignment 1/Data')
#df = pd.read_csv('/content/drive/MyDrive/Uni2025/IFN850 Machine Learning/Assignment 1/Data/kick.csv')
df = pd.read_csv(r'C:\Users\stevi\Downloads\kick.csv')
olddf = df.copy()

In [ ]:
# Print Dataset
print(df.info())
display(df)

In [ ]:
# current vs expected data types

# #   Column                             Non-Null Count  Dtype
#---  ------                             --------------  -----
# 0   PurchaseID                         41476 non-null  int64   Quantitative (Numeric): Discrete > int64 (primary key, unique)
# 1   PurchaseTimestamp                  41476 non-null  int64   Quantitative (Numeric): Discrete > int64 (Unix timestamp, can be converted to datetime if needed)
# 2   PurchaseDate                       41476 non-null  object  Qualitative (Categorical): Datetime > object (needs conversion to datetime)
# 3   Auction                            41432 non-null  object  Qualitative (Categorical): Nominal > object
# 4   VehYear                            41432 non-null  float64 Quantitative (Numeric): Discrete > int64 (needs conversion to int64 after cleaning)
# 5   Make                               41432 non-null  object  Qualitative (Categorical): Nominal > object
# 6   Color                              41432 non-null  object  Qualitative (Categorical): Nominal > object
# 7   Transmission                       41432 non-null  object  Quantitative (Numeric): Binary > int64 (requires cleaning then conversion to binary)
# 8   WheelTypeID                        41432 non-null  object  Qualitative (Categorical): Nominal > Int64 (stored as integer codes after cleaning)
# 9   WheelType                          41380 non-null  object  Qualitative (Categorical): Nominal > object
# 10  VehOdo                             41432 non-null  float64 Quantitative (Numeric): Continuous > int64 (needs conversion to int64 after cleaning)
# 11  Nationality                        41432 non-null  object  Qualitative (Categorical): Nominal > object
# 12  Size                               41432 non-null  object  Qualitative (Categorical): Nominal > object (potentially break this down into other variables to keep order)
# 13  TopThreeAmericanName               41432 non-null  object  Qualitative (Categorical): Nominal > object
# 14  MMRAcquisitionAuctionAveragePrice  41416 non-null  object  Quantitative (Numeric): Continuous > int64 (needs conversion to int64 after cleaning)
# 15  MMRAcquisitionAuctionCleanPrice    41429 non-null  object  Quantitative (Numeric): Continuous > int64 (needs conversion to int64 after cleaning)
# 16  MMRAcquisitionRetailAveragePrice   41429 non-null  object  Quantitative (Numeric): Continuous > int64 (needs conversion to int64 after cleaning)
# 17  MMRAcquisitonRetailCleanPrice      41327 non-null  object  Quantitative (Numeric): Continuous > int64 (needs conversion to int64 after cleaning)
# 18  MMRCurrentAuctionAveragePrice      41429 non-null  object  Quantitative (Numeric): Continuous > int64 (needs conversion to int64 after cleaning)
# 19  MMRCurrentAuctionCleanPrice        41429 non-null  object  Quantitative (Numeric): Continuous > int64 (needs conversion to int64 after cleaning)
# 20  MMRCurrentRetailAveragePrice       41409 non-null  object  Quantitative (Numeric): Continuous > int64 (needs conversion to int64 after cleaning)
# 21  MMRCurrentRetailCleanPrice         41409 non-null  object  Quantitative (Numeric): Continuous > int64 (needs conversion to int64 after cleaning)
# 22  MMRCurrentRetailRatio              41116 non-null  object  Quantitative (Numeric): Continuous > float64 (needs conversion to float64 after cleaning)
# 23  PRIMEUNIT                          41432 non-null  object  Quantitative (Numeric): Binary > int64 (requires cleaning then conversion to binary)
# 24  AUCGUART                           41432 non-null  object  Quantitative (Numeric): Binary > int64 (requires cleaning then conversion to binary)
# 25  VNST                               41432 non-null  object  Qualitative (Categorical): Nominal > object
# 26  VehBCost                           41432 non-null  object  Quantitative (Numeric): Continuous > int64 (needs conversion to int64 after cleaning)
# 27  IsOnlineSale                       41432 non-null  object  Quantitative (Numeric): Binary > int64 (requires cleaning then conversion to binary)
# 28  WarrantyCost                       41432 non-null  float64 Quantitative (Numeric): Continuous > int64 (needs conversion to int64 after cleaning)
# 29  ForSale                            41476 non-null  object  Quantitative (Numeric): Binary > int64 (requires cleaning then conversion to binary)
# 30  IsBadBuy                           41476 non-null  int64   Quantitative (Numeric): Binary > int64 (target variable)

In [ ]:
# Function to inspect Qualitative variable with simple plot and basic statistics
def inspect_qualitative_variable(df, column):
    print(f"\n=== {column} ===")

    # Calculate basic statistics
    total = len(df)                                  # Total number of records
    missing = df[column].isnull().sum()              # Count of missing (NaN) values
    unique_vals = df[column].nunique(dropna=True)    # Number of unique categories (excluding NaN)

    # Print summary statistics
    print(f"Total values       : {total}")
    print(f"Missing values     : {missing}")
    print(f"Unique categories  : {unique_vals}")
    print("\nValue counts:")
    print(df[column].value_counts(dropna=False))     # Print count of each category including NaN

    # Plot count distribution as a bar chart
    plt.figure(figsize=(8, 5))                        # Set plot size
    sns.countplot(
        data=df,
        x=column,
        hue=column,                                   # Apply color separation by the same variable
        order=df[column].value_counts().index,        # Display bars in descending order of frequency
        palette='pastel'                              # Use a pastel color palette
    )
    plt.title(f'{column} Distribution')               # Set plot title
    plt.xlabel(column.capitalize())                   # Label x-axis with column name
    plt.ylabel('Count')                               # Label y-axis
    plt.xticks(rotation=45)                           # Rotate x-axis labels for readability
    plt.tight_layout()                                # Adjust layout to prevent overlap
    plt.show()                                        # Display the plot

In [ ]:
# Define a function to inspect a Quantitative variable in a DataFrame
def inspect_quantitative_variable(df, column):
    print(f"\n=== {column} ===")

    # Calculate basic statistics
    total = len(df)                                  # Total number of entries
    missing = df[column].isnull().sum()              # Count of missing values
    negative = (df[column] < 0).sum()                # Count of negative values
    zero = (df[column] == 0).sum()                   # Count of zero values
    min_val = df[column].min()                       # Minimum value
    max_val = df[column].max()                       # Maximum value
    mean = df[column].mean()                         # Mean of the column
    median = df[column].median()                     # Median of the column
    std = df[column].std()                           # Standard deviation

    # Print the calculated statistics
    print(f"Total values     : {total}")
    print(f"Missing values   : {missing}")
    print(f"Negative values  : {negative}")
    print(f"Zero values      : {zero}")
    print(f"Min              : {min_val}")
    print(f"Max              : {max_val}")
    print(f"Mean             : {mean:.2f}")
    print(f"Median           : {median:.2f}")
    print(f"Std. Deviation   : {std:.2f}")

    # Visualise the distribution of values using a histogram and boxplot
    plt.figure(figsize=(12, 4))                      # Set up a wide figure for side-by-side plots

    # Plot histogram with KDE (kernel density estimate)
    plt.subplot(1, 2, 1)
    sns.histplot(df[column], kde=True, bins='auto', color='steelblue')  # Automatically choose bin size
    plt.title(f'Histogram of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')

    # Plot boxplot to show spread and potential outliers
    plt.subplot(1, 2, 2)
    sns.boxplot(x=df[column], color='salmon')
    plt.title(f'Boxplot of {column}')
    plt.xlabel(column)

    # Adjust layout to prevent overlap and display the plots
    plt.tight_layout()
    plt.show()


# Data Cleaning

In [ ]:
# During some investigative work in Excel I noticed these columns all contain 44 NaN values.
# Since imputing values across so many features would effectively create “synthetic” vehicle records
# that do not reflect real data, I have decided to remove all 44 NaN records.

cols_to_check = [
    'Auction', 'VehYear', 'Make', 'Color', 'Transmission',
    'WheelTypeID', 'WheelType', 'VehOdo', 'Nationality',
    'Size', 'TopThreeAmericanName',
    'MMRAcquisitionAuctionAveragePrice', 'MMRAcquisitionAuctionCleanPrice',
    'MMRAcquisitionRetailAveragePrice', 'MMRAcquisitonRetailCleanPrice',
    'MMRCurrentAuctionAveragePrice', 'MMRCurrentAuctionCleanPrice',
    'MMRCurrentRetailAveragePrice', 'MMRCurrentRetailCleanPrice',
    'MMRCurrentRetailRatio', 'VehBCost'
]

# Drop rows where all of these columns are NaN (the 44 fully missing records)
df = df.dropna(subset=cols_to_check, how='all')


# Additionally, 6 rows were found where multiple categorical variables
# (Color, Transmission, WheelTypeID, WheelType, PRIMEUNIT, AUCGUART)
# contained only the placeholder value "?" instead of real data.
# Since imputing across many features would again create artificial vehicle records

cols_with_question = ['Color', 'Transmission', 'WheelTypeID', 'WheelType', 'PRIMEUNIT', 'AUCGUART']

rows_to_drop = df.index[df[cols_with_question].eq('?').all(axis=1)]
df = df.drop(rows_to_drop)

## PurchaseID

In [ ]:
# show unique and value_counts
print(df['PurchaseID'].unique())
print(df['PurchaseID'].value_counts())

In [ ]:
# check is there is any missing values
df['PurchaseID'].isna().sum()

In [ ]:
# show min/max values
df['PurchaseID'].min(), df['PurchaseID'].max()

## PurchaseTimestamp

In [ ]:
# show unique and value_counts
print(df['PurchaseTimestamp'].unique())
print(df['PurchaseTimestamp'].value_counts())

In [ ]:
# check is there is any missing values
df['PurchaseTimestamp'].isna().sum()

In [ ]:
# show min/max values
df['PurchaseTimestamp'].min(), df['PurchaseTimestamp'].max()

In [ ]:
# Check if any dates are invalid after converting to datetime
pdtdf = df.copy()
pdtdf['PurchaseDatetime'] = pd.to_datetime(df['PurchaseTimestamp'], unit='s', errors='coerce')
pdtdf['PurchaseDatetime'].isna().sum()

In [ ]:
pdtdf['PurchaseDatetime'].describe()

## PurchaseDate

In [ ]:
# show unique and value_counts
print(df['PurchaseDate'].unique())
print(df['PurchaseDate'].value_counts())

In [ ]:
# check is there is any missing values
df['PurchaseDate'].isna().sum()

In [ ]:
# show min/max values
df['PurchaseDate'].min(), df['PurchaseDate'].max()

In [ ]:
# Check if any dates are invalid after converting to datetime
df['PurchaseDate'] = pd.to_datetime(df['PurchaseDate'], errors='coerce', dayfirst=True)
df['PurchaseDate'].isna().sum()

# create new month column
# year isnt particulary needed because we have VehYear as a show of age for the vehicle
df['Month'] = df['PurchaseDate'].dt.month

## Auction

In [ ]:
# show unique and value_counts
print(df['Auction'].unique())
print(df['Auction'].value_counts())

In [ ]:
inspect_qualitative_variable(df, 'Auction')

## VehYear

Vehicle make year

In [ ]:
# show unique and value_counts
print(df['VehYear'].unique())
print(df['VehYear'].value_counts())

In [ ]:
inspect_quantitative_variable(df, 'VehYear')

In [ ]:
df['VehYear'] = df['VehYear'].astype('int64')

## Make

In [ ]:
# show unique and value_counts
print(df['Make'].unique())
print(df['Make'].value_counts())

In [ ]:
inspect_qualitative_variable(df, 'Make')

## Color

In [ ]:
# show unique
print(df['Color'].unique())


In [ ]:
inspect_qualitative_variable(df, 'Color')

In [ ]:
# Given the small count of NOT AVAIL, it was merged into the OTHER category
df.loc[df['Color'] == 'NOT AVAIL', 'Color'] = 'OTHER'

In [ ]:
inspect_qualitative_variable(df, 'Color')

## Transmission

In [ ]:
# show unique and value_counts
print(df['Transmission'].unique())

In [ ]:
inspect_qualitative_variable(df, 'Transmission')

In [ ]:
# Convert Manual into MANUAL
df.loc[df['Transmission'] == 'Manual', 'Transmission'] = 'MANUAL'

In [ ]:
# Convert to binary 0,1 variable
df['Transmission'] = df['Transmission'].map({'AUTO': 0, 'MANUAL': 1})

In [ ]:
inspect_qualitative_variable(df, 'Transmission')

## WheelTypeID

In [ ]:
# show unique and value_counts
print(df['WheelTypeID'].unique())

In [ ]:
inspect_qualitative_variable(df, 'WheelTypeID')

In [ ]:
# Contains invalid values coded as ? and 0, since these dont correspond to a wheel type from WheelType I have decided to impute them with the Mode
# replace '?' with NaN
df['WheelTypeID'] = df['WheelTypeID'].replace(['?', 0], pd.NA)

In [ ]:
# fill with mode
df['WheelTypeID'] = df['WheelTypeID'].fillna(df['WheelTypeID'].mode()[0])

In [ ]:
# convert to integer
df['WheelTypeID'] = df['WheelTypeID'].astype('Int64')

## WheelType

In [ ]:
# “WheelType and WheelTypeID represent the same information in different formats.
# To avoid inconsistency after imputation, WheelTypeID was cleaned and imputed using the mode.
# WheelType was then regenerated directly from WheelTypeID using a fixed mapping (1 = Alloy, 2 = Covers, 3 = Special).
# This ensures both columns remain consistent.”

wheeltype_map = {
    1: "Alloy",
    2: "Covers",
    3: "Special"
}

df['WheelType'] = df['WheelTypeID'].map(wheeltype_map)

In [ ]:
inspect_qualitative_variable(df, 'WheelType')

## VehOdo

In [ ]:
# show unique and value_counts
print(df['WheelTypeID'].unique())

In [ ]:
inspect_quantitative_variable(df, 'VehOdo')

In [ ]:
# convert to integer
df['VehOdo'] = df['VehOdo'].astype('int64')

## Nationality

In [ ]:
# show unique and value_counts
print(df['Nationality'].unique())

In [ ]:
inspect_qualitative_variable(df, 'Nationality')

In [ ]:
# replace '?' with NaN
df['Nationality'] = df['Nationality'].replace('?', pd.NA)

In [ ]:
# fill with mode
df['WheelTypeID'] = df['WheelTypeID'].fillna(df['WheelTypeID'].mode()[0])

## Size + CarSize SUVSize TruckSize

In [ ]:
# show unique and value_counts
print(df['Size'].unique())

In [ ]:
inspect_qualitative_variable(df, 'Size')

In [ ]:
# replace '?' with NaN
df['Size'] = df['Size'].replace('?', pd.NA)

In [ ]:
# fill with mode
df['Size'] = df['Size'].fillna(df['Size'].mode()[0])

## TopThreeAmericanName

In [ ]:
# show unique and value_counts
print(df['TopThreeAmericanName'].unique())

In [ ]:
inspect_qualitative_variable(df, 'TopThreeAmericanName')

In [ ]:
# replace '?' with NaN
df['Size'] = df['Size'].replace('?', pd.NA)

In [ ]:
# fill with mode
df['Size'] = df['Size'].fillna(df['Size'].mode()[0])

Ready for Onehot encoding now

## MMRAcquisitionAuctionAveragePrice

In [ ]:
# show unique and value_counts
print(df['MMRAcquisitionAuctionAveragePrice'].unique())
print(df['MMRAcquisitionAuctionAveragePrice'].value_counts())

In [ ]:
# Shows as an object when its meant to be a numeric column
bad_mask = pd.to_numeric(df['MMRAcquisitionAuctionAveragePrice'], errors='coerce').isna()
print(df.loc[bad_mask, 'MMRAcquisitionAuctionAveragePrice'].value_counts())
pd.to_numeric(df['MMRAcquisitionAuctionAveragePrice'], errors='coerce').isna().sum()

In [ ]:
# after inspecting in excel and python it has some NaN and ? values
# replace '?' with NaN
df['MMRAcquisitionAuctionAveragePrice'] = df['MMRAcquisitionAuctionAveragePrice'].replace('?', pd.NA)

# convert to numeric
df['MMRAcquisitionAuctionAveragePrice'] = pd.to_numeric(df['MMRAcquisitionAuctionAveragePrice'])

In [ ]:
inspect_quantitative_variable(df, 'MMRAcquisitionAuctionAveragePrice')

In [ ]:
# replace '0' with NaN as 0 is technically a blank entry
df['MMRAcquisitionAuctionAveragePrice'] = df['MMRAcquisitionAuctionAveragePrice'].replace([0], pd.NA)

# finally for this variable there are many "0" so I need to impute them with the median as the data is quite skewed
df['MMRAcquisitionAuctionAveragePrice'] = df['MMRAcquisitionAuctionAveragePrice'].fillna(
    df['MMRAcquisitionAuctionAveragePrice'].median()
).astype('Int64')

In [ ]:
# Inspect after imputing for median
inspect_quantitative_variable(df, 'MMRAcquisitionAuctionAveragePrice')

## MMRAcquisitionAuctionCleanPrice

In [ ]:
# show unique and value_counts
print(df['MMRAcquisitionAuctionCleanPrice'].unique())
print(df['MMRAcquisitionAuctionCleanPrice'].value_counts())

In [ ]:
# Shows as an object when its meant to be a numeric column
bad_mask = pd.to_numeric(df['MMRAcquisitionAuctionCleanPrice'], errors='coerce').isna()
print(df.loc[bad_mask, 'MMRAcquisitionAuctionCleanPrice'].value_counts())
pd.to_numeric(df['MMRAcquisitionAuctionCleanPrice'], errors='coerce').isna().sum()

In [ ]:
# after inspecting in excel and python it has some NaN and ? values
# replace '?' with NaN
df['MMRAcquisitionAuctionCleanPrice'] = df['MMRAcquisitionAuctionCleanPrice'].replace('?', pd.NA)

# convert to numeric
df['MMRAcquisitionAuctionCleanPrice'] = pd.to_numeric(df['MMRAcquisitionAuctionCleanPrice'])

In [ ]:
inspect_quantitative_variable(df, 'MMRAcquisitionAuctionCleanPrice')

In [ ]:
# replace '0,1' with NaN as 0 & 1 is technically missing data
df['MMRAcquisitionAuctionCleanPrice'] = df['MMRAcquisitionAuctionCleanPrice'].replace([0,1], pd.NA)

# finally for this variable there are many "0" so I need to impute them with the median as the data is quite skewed
df['MMRAcquisitionAuctionCleanPrice'] = df['MMRAcquisitionAuctionCleanPrice'].fillna(
    df['MMRAcquisitionAuctionCleanPrice'].median()
).astype('Int64')

In [ ]:
inspect_quantitative_variable(df, 'MMRAcquisitionAuctionCleanPrice')

Not sure about the outlier of 1's in this documentation says - Acquisition price for this vehicle in the above condition as of the current day

## MMRAcquisitionRetailAveragePrice

In [ ]:
# show unique and value_counts
print(df['MMRAcquisitionRetailAveragePrice'].unique())
print(df['MMRAcquisitionRetailAveragePrice'].value_counts())

In [ ]:
# Shows as an object when its meant to be a numeric column
bad_mask = pd.to_numeric(df['MMRAcquisitionRetailAveragePrice'], errors='coerce').isna()
print(df.loc[bad_mask, 'MMRAcquisitionRetailAveragePrice'].value_counts())
pd.to_numeric(df['MMRAcquisitionRetailAveragePrice'], errors='coerce').isna().sum()

In [ ]:
# after inspecting in excel and python it has some NaN and ? values
# replace '?' with NaN
df['MMRAcquisitionRetailAveragePrice'] = df['MMRAcquisitionRetailAveragePrice'].replace('?', pd.NA)

# convert to numeric
df['MMRAcquisitionRetailAveragePrice'] = pd.to_numeric(df['MMRAcquisitionRetailAveragePrice'])

In [ ]:
inspect_quantitative_variable(df, 'MMRAcquisitionRetailAveragePrice')

In [ ]:
# replace '0' with NaN as 0 is technically a blank entry
df['MMRAcquisitionRetailAveragePrice'] = df['MMRAcquisitionRetailAveragePrice'].replace([0], pd.NA)

# finally for this variable there are many "0" so I need to impute them with the median as the data is quite skewed
df['MMRAcquisitionRetailAveragePrice'] = (
    df['MMRAcquisitionRetailAveragePrice']
    .fillna(round(df['MMRAcquisitionRetailAveragePrice'].median()))
    .astype('int64')
)

In [ ]:
inspect_quantitative_variable(df, 'MMRAcquisitionRetailAveragePrice')

## MMRAcquisitonRetailCleanPrice

In [ ]:
# show unique and value_counts
print(df['MMRAcquisitonRetailCleanPrice'].unique())
print(df['MMRAcquisitonRetailCleanPrice'].value_counts())

In [ ]:
# Shows as an object when its meant to be a numeric column
bad_mask = pd.to_numeric(df['MMRAcquisitonRetailCleanPrice'], errors='coerce').isna()
print(df.loc[bad_mask, 'MMRAcquisitonRetailCleanPrice'].value_counts())
pd.to_numeric(df['MMRAcquisitonRetailCleanPrice'], errors='coerce').isna().sum()

In [ ]:
# after inspecting in excel and python it has some NaN and ? values
# replace '?' with NaN
df['MMRAcquisitonRetailCleanPrice'] = df['MMRAcquisitonRetailCleanPrice'].replace('?', pd.NA)

# convert to numeric
df['MMRAcquisitonRetailCleanPrice'] = pd.to_numeric(df['MMRAcquisitonRetailCleanPrice'])

In [ ]:
inspect_quantitative_variable(df, 'MMRAcquisitonRetailCleanPrice')

In [ ]:
# replace '0' with NaN as 0 is technically a blank entry
df['MMRAcquisitonRetailCleanPrice'] = df['MMRAcquisitonRetailCleanPrice'].replace([0], pd.NA)

# finally for this variable there are many "0" so I need to impute them with the median as the data is quite skewed
df['MMRAcquisitonRetailCleanPrice'] = df['MMRAcquisitonRetailCleanPrice'].fillna(
    df['MMRAcquisitonRetailCleanPrice'].median()
).astype('Int64')

In [ ]:
inspect_quantitative_variable(df, 'MMRAcquisitonRetailCleanPrice')

## MMRCurrentAuctionAveragePrice

In [ ]:
# show unique and value_counts
print(df['MMRCurrentAuctionAveragePrice'].unique())
print(df['MMRCurrentAuctionAveragePrice'].value_counts())

In [ ]:
# Shows as an object when its meant to be a numeric column
bad_mask = pd.to_numeric(df['MMRCurrentAuctionAveragePrice'], errors='coerce').isna()
print(df.loc[bad_mask, 'MMRCurrentAuctionAveragePrice'].value_counts())
pd.to_numeric(df['MMRCurrentAuctionAveragePrice'], errors='coerce').isna().sum()

In [ ]:
# after inspecting in excel and python it has some NaN and ? values
# replace '?' with NaN
df['MMRCurrentAuctionAveragePrice'] = df['MMRCurrentAuctionAveragePrice'].replace('?', pd.NA)

# convert to numeric
df['MMRCurrentAuctionAveragePrice'] = pd.to_numeric(df['MMRCurrentAuctionAveragePrice'])

In [ ]:
inspect_quantitative_variable(df, 'MMRCurrentAuctionAveragePrice')

In [ ]:
# replace '0' with NaN as 0 is technically a blank entry
df['MMRCurrentAuctionAveragePrice'] = df['MMRCurrentAuctionAveragePrice'].replace([0], pd.NA)

# finally for this variable there are many "0" so I need to impute them with the median as the data is quite skewed
df['MMRCurrentAuctionAveragePrice'] = (
    df['MMRCurrentAuctionAveragePrice']
    .fillna(round(df['MMRCurrentAuctionAveragePrice'].median()))
    .astype('int64')
)

In [ ]:
inspect_quantitative_variable(df, 'MMRCurrentAuctionAveragePrice')

## MMRCurrentAuctionCleanPrice

In [ ]:
# show unique and value_counts
print(df['MMRCurrentAuctionCleanPrice'].unique())
print(df['MMRCurrentAuctionCleanPrice'].value_counts())

In [ ]:
# Shows as an object when its meant to be a numeric column
bad_mask = pd.to_numeric(df['MMRCurrentAuctionCleanPrice'], errors='coerce').isna()
print(df.loc[bad_mask, 'MMRCurrentAuctionCleanPrice'].value_counts())
pd.to_numeric(df['MMRCurrentAuctionCleanPrice'], errors='coerce').isna().sum()

In [ ]:
# after inspecting in excel and python it has some NaN and ? values
# replace '?' with NaN
df['MMRCurrentAuctionCleanPrice'] = df['MMRCurrentAuctionCleanPrice'].replace('?', pd.NA)

# convert to numeric
df['MMRCurrentAuctionCleanPrice'] = pd.to_numeric(df['MMRCurrentAuctionCleanPrice'])

In [ ]:
inspect_quantitative_variable(df, 'MMRCurrentAuctionCleanPrice')

In [ ]:
# replace '0,1' with NaN as 0 & 1 is technically missing data
df['MMRCurrentAuctionCleanPrice'] = df['MMRCurrentAuctionCleanPrice'].replace([0, 1], pd.NA)

# finally for this variable there are many "0" so I need to impute them with the median as the data is quite skewed
df['MMRCurrentAuctionCleanPrice'] = df['MMRCurrentAuctionCleanPrice'].fillna(
    df['MMRCurrentAuctionCleanPrice'].median()
).astype('Int64')

In [ ]:
inspect_quantitative_variable(df, 'MMRCurrentAuctionCleanPrice')

## MMRCurrentRetailAveragePrice

In [ ]:
# show unique and value_counts
print(df['MMRCurrentRetailAveragePrice'].unique())
print(df['MMRCurrentRetailAveragePrice'].value_counts())

In [ ]:
# Shows as an object when its meant to be a numeric column
bad_mask = pd.to_numeric(df['MMRCurrentRetailAveragePrice'], errors='coerce').isna()
print(df.loc[bad_mask, 'MMRCurrentRetailAveragePrice'].value_counts())
pd.to_numeric(df['MMRCurrentRetailAveragePrice'], errors='coerce').isna().sum()

In [ ]:
# after inspecting in excel and python it has some NaN and ? values
# replace '?' with NaN
df['MMRCurrentRetailAveragePrice'] = df['MMRCurrentRetailAveragePrice'].replace('?', pd.NA)

# convert to numeric
df['MMRCurrentRetailAveragePrice'] = pd.to_numeric(df['MMRCurrentRetailAveragePrice'])

In [ ]:
inspect_quantitative_variable(df, 'MMRCurrentRetailAveragePrice')

In [ ]:
# replace '0' with NaN as 0 is technically a blank entry
df['MMRCurrentRetailAveragePrice'] = df['MMRCurrentRetailAveragePrice'].replace([0], pd.NA)

# finally for this variable there are many "0" so I need to impute them with the median as the data is quite skewed
df['MMRCurrentRetailAveragePrice'] = (
    df['MMRCurrentRetailAveragePrice']
    .fillna(round(df['MMRCurrentRetailAveragePrice'].median()))
    .astype('int64')
)

In [ ]:
inspect_quantitative_variable(df, 'MMRCurrentRetailAveragePrice')

## MMRCurrentRetailCleanPrice

In [ ]:
# show unique and value_counts
print(df['MMRCurrentRetailCleanPrice'].unique())
print(df['MMRCurrentRetailCleanPrice'].value_counts())

In [ ]:
# Shows as an object when its meant to be a numeric column
bad_mask = pd.to_numeric(df['MMRCurrentRetailCleanPrice'], errors='coerce').isna()
print(df.loc[bad_mask, 'MMRCurrentRetailCleanPrice'].value_counts())
pd.to_numeric(df['MMRCurrentRetailCleanPrice'], errors='coerce').isna().sum()

In [ ]:
# after inspecting in excel and python it has some NaN and ? values
# replace '?' with NaN
df['MMRCurrentRetailCleanPrice'] = df['MMRCurrentRetailCleanPrice'].replace('?', pd.NA)

# convert to numeric
df['MMRCurrentRetailCleanPrice'] = pd.to_numeric(df['MMRCurrentRetailCleanPrice'])

In [ ]:
inspect_quantitative_variable(df, 'MMRCurrentRetailCleanPrice')

In [ ]:
# replace '0' with NaN as 0 is technically a blank entry
df['MMRCurrentRetailCleanPrice'] = df['MMRCurrentRetailCleanPrice'].replace([0], pd.NA)

# finally for this variable there are many "0" so I need to impute them with the median as the data is quite skewed
df['MMRCurrentRetailCleanPrice'] = df['MMRCurrentRetailCleanPrice'].fillna(
    df['MMRCurrentRetailCleanPrice'].median()
).astype('Int64')

In [ ]:
inspect_quantitative_variable(df, 'MMRCurrentRetailCleanPrice')

## MMRCurrentRetailRatio

In [ ]:
# show unique and value_counts
print(df['MMRCurrentRetailRatio'].unique())
print(df['MMRCurrentRetailRatio'].value_counts())

In [ ]:
# Shows as an object when its meant to be a numeric column
bad_mask = pd.to_numeric(df['MMRCurrentRetailRatio'], errors='coerce').isna()
print(df.loc[bad_mask, 'MMRCurrentRetailRatio'].value_counts())
pd.to_numeric(df['MMRCurrentRetailRatio'], errors='coerce').isna().sum()

In [ ]:
# after inspecting in excel and python it has some NaN and #VALUE! values
# replace '#VALUE!' with NaN
df['MMRCurrentRetailRatio'] = df['MMRCurrentRetailRatio'].replace('#VALUE!', pd.NA)

# convert to numeric
df['MMRCurrentRetailRatio'] = pd.to_numeric(df['MMRCurrentRetailRatio'])

In [ ]:
inspect_quantitative_variable(df, 'MMRCurrentRetailRatio')

In [ ]:
# finally for this variable there are many missing values so I need to impute them with the median as the data is quite skewed
df['MMRCurrentRetailRatio'] = df['MMRCurrentRetailRatio'].fillna(
    df['MMRCurrentRetailRatio'].median()
)

In [ ]:
inspect_quantitative_variable(df, 'MMRCurrentRetailRatio')

## PRIMEUNIT

In [ ]:
# show unique and value_counts
print(df['PRIMEUNIT'].unique())

In [ ]:
inspect_qualitative_variable(df, 'PRIMEUNIT')

In [ ]:
# replace '?' with NaN
df['PRIMEUNIT'] = df['PRIMEUNIT'].replace('?', pd.NA)

df['PRIMEUNIT'] = df['PRIMEUNIT'].fillna(df['PRIMEUNIT'].mode()[0])

In [ ]:
# Convert to binary 0,1 variable
df['PRIMEUNIT'] = df['PRIMEUNIT'].map({'NO': 0, 'YES': 1})

In [ ]:
inspect_qualitative_variable(df, 'PRIMEUNIT')

## AUCGUART

In [ ]:
# show unique and value_counts
print(df['AUCGUART'].unique())

In [ ]:
inspect_qualitative_variable(df, 'AUCGUART')

In [ ]:
# AUCGUART has over 90% '?' values. imputing with mode (GREEN) would skew distribution. Instead keeping '?' as 'UNKNOWN' to preserve the info without bias.
df['AUCGUART'] = df['AUCGUART'].replace('?', 'UNKNOWN')

In [ ]:
inspect_qualitative_variable(df, 'AUCGUART')

## VNST

In [ ]:
# show unique and value_counts
print(df['VNST'].unique())

In [ ]:
inspect_qualitative_variable(df, 'VNST')

## VehBCost

In [ ]:
# show unique and value_counts
print(df['VehBCost'].unique())
print(df['VehBCost'].value_counts())

In [ ]:
# Shows as an object when its meant to be a numeric column
bad_mask = pd.to_numeric(df['VehBCost'], errors='coerce').isna()
print(df.loc[bad_mask, 'VehBCost'].value_counts())
pd.to_numeric(df['VehBCost'], errors='coerce').isna().sum()

In [ ]:
# after inspecting in excel and python it has some NaN and ? values
# replace '?' with NaN
df['VehBCost'] = df['VehBCost'].replace('?', pd.NA)

# convert to numeric
df['VehBCost'] = pd.to_numeric(df['VehBCost'])

In [ ]:
inspect_quantitative_variable(df, 'VehBCost')

In [ ]:
# finally for this variable there are afew missing entries so I need to impute them with the median as the data is quite skewed
df['VehBCost'] = df['VehBCost'].fillna(
    df['VehBCost'].median()
).astype('int64')

In [ ]:
# Inspect after imputing for median
inspect_quantitative_variable(df, 'VehBCost')

## IsOnlineSale

In [ ]:
# show unique and value_counts
print(df['IsOnlineSale'].unique())
print(df['IsOnlineSale'].value_counts())

In [ ]:
# Shows as an object when its meant to be a numeric column
bad_mask = pd.to_numeric(df['IsOnlineSale'], errors='coerce').isna()
print(df.loc[bad_mask, 'IsOnlineSale'].value_counts())
pd.to_numeric(df['IsOnlineSale'], errors='coerce').isna().sum()

In [ ]:
# after inspecting in excel and python it has some NaN and ? values
# replace '?' with NaN
df['IsOnlineSale'] = df['IsOnlineSale'].replace('?', pd.NA)

# convert to numeric
df['IsOnlineSale'] = pd.to_numeric(df['IsOnlineSale'])

In [ ]:
inspect_qualitative_variable(df, 'IsOnlineSale')

In [ ]:
# replace variables with NaN
df['IsOnlineSale'] = df['IsOnlineSale'].replace(['?',-1,2.0,4.0], pd.NA)

# round & convert to integer
df['IsOnlineSale'] = df['IsOnlineSale'].round().astype('Int64')

# convert remaining nan to mode
df['IsOnlineSale'] = df['IsOnlineSale'].fillna(df['IsOnlineSale'].mode()[0])

In [ ]:
inspect_qualitative_variable(df, 'IsOnlineSale')

## WarrantyCost

In [ ]:
# show unique and value_counts
print(df['WarrantyCost'].unique())
print(df['WarrantyCost'].value_counts())

In [ ]:
# Shows as an object when its meant to be a numeric column
bad_mask = pd.to_numeric(df['WarrantyCost'], errors='coerce').isna()
print(df.loc[bad_mask, 'WarrantyCost'].value_counts())
pd.to_numeric(df['WarrantyCost'], errors='coerce').isna().sum()

In [ ]:
df['WarrantyCost'] = df['WarrantyCost'].astype('int64')

In [ ]:
inspect_quantitative_variable(df, 'WarrantyCost')

No cleaning required - ready for one hot encoding

## ForSale

In [ ]:
# show unique and value_counts
print(df['ForSale'].unique())

In [ ]:
inspect_qualitative_variable(df, 'ForSale')

In [ ]:
# converting other types to yes and no to respective values
df['ForSale'] = df['ForSale'].replace({
    'Yes': 'YES',
    'yes': 'YES',
    '0': pd.NA,   # treating this as a missing variable
    'No': 'NO',
    '?': pd.NA
})

# convert to binary 0,1 variable
df['ForSale'] = df['ForSale'].map({'NO': 0, 'YES': 1})

# impute missing with mode (most likely YES)
df['ForSale'] = df['ForSale'].fillna(df['ForSale'].mode()[0])

# convert to int
df['ForSale'] = df['ForSale'].astype('int64')

In [ ]:
inspect_qualitative_variable(df, 'ForSale')

## IsBadBuy

In [ ]:
# show unique and value_counts
print(df['WarrantyCost'].unique())
print(df['WarrantyCost'].value_counts())

In [ ]:
inspect_qualitative_variable(df, 'IsBadBuy')

In [ ]:
# data types

# #   Column                             Non-Null Count  Dtype
#---  ------                             --------------  -----
# 0   PurchaseID                         41476 non-null  int64   Quantitative (Numeric): Discrete > int64 (primary key, unique)
# 1   PurchaseTimestamp                  41476 non-null  int64   Quantitative (Numeric): Interval? > int64 (Unix timestamp, can be converted to datetime if needed)
# 2   PurchaseDate                       41476 non-null  object  Qualitative (Categorical): Datetime > object
# 3   Auction                            41432 non-null  object  Qualitative (Categorical): Nominal > object
# 4   VehYear                            41432 non-null  float64 Quantitative (Numeric): Discrete/Interval > int64
# 5   Make                               41432 non-null  object  Qualitative (Categorical): Nominal > object
# 6   Color                              41432 non-null  object  Qualitative (Categorical): Nominal > object
# 7   Transmission                       41432 non-null  object  Quantitative (Numeric): Binary > int64
# 8   WheelTypeID                        41432 non-null  object  Qualitative (Categorical): Nominal > Int64
# 9   WheelType                          41380 non-null  object  Qualitative (Categorical): Nominal > object
# 10  VehOdo                             41432 non-null  float64 Quantitative (Numeric): Continuous/Ratio > int64
# 11  Nationality                        41432 non-null  object  Qualitative (Categorical): Nominal > object
# 12  Size                               41432 non-null  object  Qualitative (Categorical): Nominal > object
# 13  TopThreeAmericanName               41432 non-null  object  Qualitative (Categorical): Nominal > object
# 14  MMRAcquisitionAuctionAveragePrice  41416 non-null  object  Quantitative (Numeric): Continuous/Ratio > int64
# 15  MMRAcquisitionAuctionCleanPrice    41429 non-null  object  Quantitative (Numeric): Continuous/Ratio > int64
# 16  MMRAcquisitionRetailAveragePrice   41429 non-null  object  Quantitative (Numeric): Continuous/Ratio > int64
# 17  MMRAcquisitonRetailCleanPrice      41327 non-null  object  Quantitative (Numeric): Continuous/Ratio > int64
# 18  MMRCurrentAuctionAveragePrice      41429 non-null  object  Quantitative (Numeric): Continuous/Ratio > int64
# 19  MMRCurrentAuctionCleanPrice        41429 non-null  object  Quantitative (Numeric): Continuous/Ratio > int64
# 20  MMRCurrentRetailAveragePrice       41409 non-null  object  Quantitative (Numeric): Continuous/Ratio > int64
# 21  MMRCurrentRetailCleanPrice         41409 non-null  object  Quantitative (Numeric): Continuous/Ratio > int64
# 22  MMRCurrentRetailRatio              41116 non-null  object  Quantitative (Numeric): Continuous/Ratio > float64
# 23  PRIMEUNIT                          41432 non-null  object  Quantitative (Numeric): Binary > int64
# 24  AUCGUART                           41432 non-null  object  Qualitative (Categorical): Nominal > object
# 25  VNST                               41432 non-null  object  Qualitative (Categorical): Nominal > object
# 26  VehBCost                           41432 non-null  object  Quantitative (Numeric): Continuous/Ratio > int64
# 27  IsOnlineSale                       41432 non-null  object  Quantitative (Numeric): Binary > int64
# 28  WarrantyCost                       41432 non-null  float64 Quantitative (Numeric): Continuous/Ratio > int64
# 29  ForSale                            41476 non-null  object  Quantitative (Numeric): Binary > int64
# 30  IsBadBuy                           41476 non-null  int64   Quantitative (Numeric): Binary > int64 (target variable)
# 31  Month                              41426 non-null  int32   Quantitative (Numeric): Discrete/Ordinal > int32 (represents possible seasonal purchase patterns)


In [ ]:
# #   Column                             Non-Null Count  Dtype
#---  ------                             --------------  -----
# 0   Auction                            41432 non-null  object  Qualitative (Categorical): Nominal > object
# 1   VehYear                            41432 non-null  float64 Quantitative (Numeric): Discrete/Interval > int64
# 2   Make                               41432 non-null  object  Qualitative (Categorical): Nominal > object
# 3   Color                              41432 non-null  object  Qualitative (Categorical): Nominal > object
# 4   Transmission                       41432 non-null  object  Quantitative (Numeric): Binary > int64
# 5   WheelType                          41380 non-null  object  Qualitative (Categorical): Nominal > object
# 6   VehOdo                             41432 non-null  float64 Quantitative (Numeric): Continuous/Ratio > int64
# 7   Nationality                        41432 non-null  object  Qualitative (Categorical): Nominal > object
# 8   Size                               41432 non-null  object  Qualitative (Categorical): Nominal > object
# 9   TopThreeAmericanName               41432 non-null  object  Qualitative (Categorical): Nominal > object
# 10  MMRCurrentRetailAveragePrice       41409 non-null  object  Quantitative (Numeric): Continuous/Ratio > int64
# 11  MMRCurrentRetailRatio              41116 non-null  object  Quantitative (Numeric): Continuous/Ratio > float64
# 12  PRIMEUNIT                          41432 non-null  object  Quantitative (Numeric): Binary > int64
# 13  AUCGUART                           41432 non-null  object  Qualitative (Categorical): Nominal > object
# 14  VNST                               41432 non-null  object  Qualitative (Categorical): Nominal > object
# 15  VehBCost                           41432 non-null  object  Quantitative (Numeric): Continuous/Ratio > int64
# 16  IsOnlineSale                       41432 non-null  object  Quantitative (Numeric): Binary > int64
# 17  WarrantyCost                       41432 non-null  float64 Quantitative (Numeric): Continuous/Ratio > int64
# 18  IsBadBuy                           41476 non-null  int64   Quantitative (Numeric): Binary > int64 (target variable)
# 19  Month                              41426 non-null  int32   Quantitative (Numeric): Discrete/Ordinal > int32 (represents possible seasonal purchase patterns)



# Pre processing

In [ ]:
# Confirm cleaned dataset and correct DTypes
print(df.info())

In [ ]:
# Select only columns with numerical values.
numerical_columns = df.select_dtypes(include=['number'])
# Display the correlation matrix.
correlation_matrix = numerical_columns.corr()
correlation_matrix

In [ ]:
# Create a mask for the upper triangle to avoid duplicate data
correlation_mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

plt.figure(figsize=(12, 10))
# annot=True tells seaborn to display the values for each cell.
# fmt=".2f" tells seaborn to round the values to 2 decimal places, otherwise it would not fit in the cells.
# We also adjust the font size so it fits in the page (annot_kws={'size': 8})
# center=0 tells seaborn that the middle point of the heatmap should be zero.
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", annot_kws={'size': 8}, cmap='coolwarm', center=0, mask=correlation_mask, square=True)
plt.show()

In [ ]:
dfcleaned = df.drop(columns=['PurchaseID', 'PurchaseTimestamp', 'PurchaseDate', 'MMRAcquisitionAuctionAveragePrice',
                             'MMRAcquisitionAuctionCleanPrice', 'MMRAcquisitionRetailAveragePrice', 'MMRAcquisitonRetailCleanPrice',
                             'MMRCurrentAuctionAveragePrice', 'MMRCurrentAuctionCleanPrice', 'MMRCurrentRetailCleanPrice', 'Color',
                             'WheelTypeID', 'ForSale'])

dfcleaned.info()

# For tree-based models: drop original Size, keep ordinal encodings - doesnt need skewness reduction so the tree dataset is ready
dftree = dfcleaned.copy()
print(dftree.dtypes)
# one-hot encoding
dftree = pd.get_dummies(dftree)

In [ ]:
print(dftree.dtypes)

## Skewness

In [ ]:
print(dfcleaned[['VehOdo','VehBCost']].dtypes)
print(dfcleaned[['VehOdo','VehBCost']].skew(numeric_only=True))

In [ ]:
def plot_skewed_columns(df, skew_thresh=0.5):

    numeric_cols = df.select_dtypes(include=[np.number]).columns

    # calculate skewness for each numeric column
    skew_vals = df[numeric_cols].skew().sort_values(ascending=False)

    # filter only skewed columns
    skewed_cols = skew_vals[abs(skew_vals) > skew_thresh]

    print("Skewed columns:")
    print(skewed_cols)

    # plot them
    n = len(skewed_cols)
    ncols = 3
    nrows = int(np.ceil(n / ncols))

    fig, axes = plt.subplots(nrows, ncols, figsize=(15, 4*nrows))
    axes = axes.flatten()

    for i, col in enumerate(skewed_cols.index):
        sns.kdeplot(df[col].dropna(), ax=axes[i], fill=True)
        axes[i].set_title(f"{col} (skew={skew_vals[col]:.2f})")

    # remove unused subplots
    for j in range(i+1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

plot_skewed_columns(dfcleaned)


we can see WarrentyCost has a large right tail with a skew value of 2 - this must be regularised, other variables with major skew are either the target variable or are binary in which case dont need to be regularised.

In [ ]:
# copy the dataframe
df_log = dfcleaned.copy()

df_log['WarrantyCost'] = df_log['WarrantyCost'].apply(lambda x: x+1)
df_log['WarrantyCost'] = df_log['WarrantyCost'].apply(np.log)

print(f"Skewness after transform: {df_log['WarrantyCost'].skew():.2f}")

# plot the distribution
plt.figure(figsize=(8,5))
sns.kdeplot(df_log['WarrantyCost'].dropna(), fill=True, color="steelblue")
plt.title("WarrantyCost after log1p transform")
plt.xlabel("Log-transformed WarrantyCost")
plt.ylabel("Density")
plt.show()

In [ ]:
# one-hot encoding
dfregression = pd.get_dummies(df_log)

# dfnn just reuses regression dataset
dfnn = dfregression.copy()

In [ ]:
# Select only columns with numerical values.
numerical_columns = dfregression.select_dtypes(include=['number'])
# Display the correlation matrix.
correlation_matrix = numerical_columns.corr()
correlation_matrix
# Create a mask for the upper triangle to avoid duplicate data
correlation_mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

plt.figure(figsize=(12, 10))
# annot=True tells seaborn to display the values for each cell.
# fmt=".2f" tells seaborn to round the values to 2 decimal places, otherwise it would not fit in the cells.
# We also adjust the font size so it fits in the page (annot_kws={'size': 8})
# center=0 tells seaborn that the middle point of the heatmap should be zero.
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", annot_kws={'size': 8}, cmap='coolwarm', center=0, mask=correlation_mask, square=True)
plt.show()

In [ ]:
# Before preprocessing
print("Before preprocessing:")
print(olddf['IsBadBuy'].value_counts())
print(olddf['IsBadBuy'].value_counts(normalize=True))


# After preprocessing
print("\nAfter preprocessing:")
print(dfcleaned['IsBadBuy'].value_counts())
print(dfcleaned['IsBadBuy'].value_counts(normalize=True))

# Decision Tree Model

In [ ]:
# Obtain a list of labels (targets) for each of the samples. We refer to this as 'y'.
y = dftree["IsBadBuy"].values
# Remove the label (target) from the samples, as this is not a feature that we want to use to predict against.
# axis=1 is used to drop the column, instead of the row.
X = dftree.drop(['IsBadBuy'], axis=1)
# Pull out the column names before grabbing the feature array, we will need it later
feature_names = X.columns
# We use X.values to get arrays for the features, instead of a DataFrame with column names etc.
X = X.values

In [ ]:
from sklearn.model_selection import train_test_split
# Create a 30% train/test split from our samples.
# We set the random_state to the constant 10 to ensure it is the same from run-to-run.
# We set the stratisfy parameter to the labels, that way we get an even selection of 0 and 1 values in our testing set.
# Otherwise, we may end up with unbalanced 0/1 labels across the two splits.
random_state = 10
test_set_size = 0.3 # 30%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_set_size,
stratify=y, random_state=random_state)
print("Size of training set:", len(X_train))
print("Size of testing set:", len(X_test))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# simple decision tree training
tree_model = DecisionTreeClassifier(random_state=random_state)
tree_model.fit(X_train, y_train)

In [ ]:
print('*****Model parameters******\n', tree_model.get_params(deep=True))
print('Number of leaves in the trained model:', tree_model.get_n_leaves())

In [ ]:
y_pred = tree_model.predict(X_test)
print(y_pred)

In [ ]:
from sklearn.metrics import classification_report
print("Training set accuracy:", tree_model.score(X_train, y_train))
print("Testing set accuracy:", tree_model.score(X_test, y_test))
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plot_tree(tree_model, feature_names=feature_names, class_names=["GoodBuy", "BadBuy"], filled=True, max_depth=1)
plt.show()

In [ ]:
def display_feature_importances(model, feature_names, features_to_display=20):
  importances = model.feature_importances_
  # Sort them out in descending order.
  indices = np.argsort(importances)
  indices = np.flip(indices, axis=0)
  # Limit to 20 features, you can leave this out to print out everything.
  indices = indices[:features_to_display]
  for i in indices:
    print(feature_names[i], ':', importances[i])
  print("Number of leaves:", model.get_n_leaves())

display_feature_importances(tree_model, feature_names)

In [ ]:
def display_feature_importances(model, feature_names, features_to_display=50):
  # Grab feature importances from the model and feature name from the original X.
  # We have to get the feature names from the original dataframe, because the feature
  # array does not include column titles.
  importances = model.feature_importances_
  # Sort them out in descending order.
  indices = np.argsort(importances)
  indices = np.flip(indices, axis=0)
  # Limit to 20 features, you can leave this out to print out everything.
  indices = indices[:features_to_display]
  for i in indices:
    print(feature_names[i], ':', importances[i])
  print("Number of leaves:", model.get_n_leaves())

display_feature_importances(tree_model, feature_names)

In [ ]:
# Check the model performance for max depth from 2-20, store into two lists that we can plot.
test_score = []
train_score = []
for max_depth in range(2, 21):
  temp_model = DecisionTreeClassifier(max_depth=max_depth, random_state=random_state)
  temp_model.fit(X_train, y_train)
  test_score.append(temp_model.score(X_test, y_test))
  train_score.append(temp_model.score(X_train, y_train))
# Plot max depth hyperparameter values vs training and test accuracy score.
plt.plot(range(2, 21), train_score, 'b', range(2, 21), test_score, 'r')
plt.xlabel('max_depth\nBlue = training acc. Red = test acc.')
plt.ylabel('accuracy')
plt.show()


In [ ]:
from sklearn.model_selection import GridSearchCV
# Define grid search CV parameters.
# after some finetuning and searching it seems to perform better lower sample leaf/sample split
params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': range(2, 10),
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'class_weight': [None, 'balanced'],
}

def perform_grid_search(X_train, y_train, X_test, y_test, params, num_folds=10):
# Instantiate the search class. Pass it the model to use (the DecisionTreeClassifier), and the number of folds.
  grid = GridSearchCV(
      param_grid=params,
      estimator=DecisionTreeClassifier(random_state=random_state),
      cv=num_folds,
      verbose=1,
      n_jobs=-1,
      scoring='f1')

  grid.fit(X_train, y_train)

  # Get best model
  best_model = grid.best_estimator_

  print("Best params:", grid.best_params_)
  print("Best CV mean score:", grid.best_score_)
  print("Train accuracy:", best_model.score(X_train, y_train))
  print("Test accuracy:", best_model.score(X_test, y_test))

  # Evaluate the best fit model.
  y_pred = grid.predict(X_test)
  print(classification_report(y_test, y_pred))

  # Display parameters of the best model.
  print(grid.best_params_)
  return best_model
model_grid = perform_grid_search(X_train, y_train, X_test, y_test, params)

In [ ]:
best_tree_model = model_grid

tree_X_train = X_train
tree_y_train = y_train
tree_X_test = X_test
tree_y_test = y_test

In [ ]:
print('Number of Nodes in the trained model:', model_grid.tree_.node_count)
print('Number of leaves in the trained model:', model_grid.get_n_leaves())

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plot_tree(model_grid, feature_names=feature_names, class_names=["GoodBuy", "BadBuy"], filled=True, max_depth=1)
plt.show()

In [ ]:
display_feature_importances(model_grid, feature_names)

In [ ]:
from sklearn.metrics import roc_auc_score
y_pred_proba_dt = tree_model.predict_proba(X_test)
y_pred_proba_dt_cv = model_grid.predict_proba(X_test)
roc_index_dt = roc_auc_score(y_test, y_pred_proba_dt[:, 1])
roc_index_dt_cv = roc_auc_score(y_test, y_pred_proba_dt_cv[:, 1])
print("ROC index on test for default tree_model:", roc_index_dt)
print("ROC index on test for grid search tree_model:", roc_index_dt_cv)

In [ ]:
from sklearn.metrics import roc_curve
fpr_dt, tpr_dt, thresholds_dt = roc_curve(y_test, y_pred_proba_dt[:,1])
fpr_dt_cv, tpr_dt_cv, thresholds_dt_cv = roc_curve(y_test, y_pred_proba_dt_cv[:,1])

In [ ]:
plt.plot(fpr_dt, tpr_dt, label='ROC Curve for default tree {:.3f}'.format(roc_index_dt),
color='red', lw=0.5)
plt.plot(fpr_dt_cv, tpr_dt_cv, label='ROC Curve for grid search {:.3f}'.format(roc_index_dt_cv), color='blue', lw=0.5)
plt.plot([0, 1], [0, 1], color='navy', lw=0.5, label='Baseline', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()


In [ ]:
print('*****Model parameters******\n', tree_model.get_params(deep=True))

In [ ]:
{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini',
 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': 10,
 'splitter': 'best'}

# Regression Model

In [ ]:
# Obtain a list of labels (targets) for each of the samples. We refer to this as 'y'.
Y = dfregression["IsBadBuy"]
# Remove the label (target) from the samples, as this is not a feature that we want to use to predict against.
X = dfregression.drop(['IsBadBuy'], axis=1)
# Create a 30% train/test split from our samples.
# We set the random_state to the constant 10 to ensure it is the same from run-to-run.
# We set the stratisfy parameter to the labels, that way we get an even selection of 0 and 1 values in our testing set.
# Otherwise, we may end up with unbalanced 0/1 labels across the two splits.
# We use X.values to get arrays for the features, instead of a DataFrame with column names etc.
random_state = 10
test_set_size = 0.3 # 30%
X_train, X_test, y_train, y_test = train_test_split(X.values, Y, test_size=test_set_size,
stratify=Y, random_state=random_state)
print("Size of training set:", len(X_train))
print("Size of testing set:", len(X_test))

In [ ]:
from sklearn.preprocessing import StandardScaler

# initialise a standard scaler object
scaler = StandardScaler()

# visualise min, max, mean and standard dev of data before scaling
print("Before scaling\n-------------")
for i in range(5):
  col = X_train[:,i]
  print("Variable #{}: min {}, max {}, mean {:.2f} and std dev {:.2f}".format(i,
min(col), max(col), np.mean(col), np.std(col)))

# learn the mean and std.dev of variables from training data
# then use the learned values to transform training data
X_train = scaler.fit_transform(X_train, y_train)
print("After scaling\n-------------")
for i in range(5):
  col = X_train[:,i]
  print("Variable #{}: min {}, max {}, mean {:.2f} and std dev {:.2f}".format(i,
min(col), max(col), np.mean(col), np.std(col)))

# use the statistic that you learned from training to transform test data
# NEVER learn from test data, this is supposed to be a set of dataset
# that the model has never seen before
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=random_state)
# fit it to training data
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report
# training and test accuracy
print("Train accuracy:", model.score(X_train, y_train))
print("Test accuracy:", model.score(X_test, y_test))
# classification report on test data
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# grid search CV
params = {
    'C': np.logspace(-3, 3, 9),
    'solver': ['lbfgs', 'newton-cg'],
    'class_weight': [None, 'balanced'],
    'max_iter': [1000],
    'tol': [1e-4, 1e-3, 1e-2]
}

# use all cores to tune logistic regression with C parameter
cv = GridSearchCV(param_grid=params,
estimator=LogisticRegression(random_state=random_state), cv=10, n_jobs=-1, scoring='f1')

cv.fit(X_train, y_train)

# test the best model
print("Train accuracy:", cv.score(X_train, y_train))
print("Test accuracy:", cv.score(X_test, y_test))

y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred))

# print parameters of the best model
print(cv.best_params_)

In [ ]:
# grab feature importances from the model and feature name from the original X
model = cv.best_estimator_
coef = model.coef_[0]
feature_names = X.columns
# sort them out in descending order
indices = np.argsort(np.absolute(coef))[::-1]
#indices = np.flip(indices, axis=0)

for i in indices:
  print(feature_names[i], ':', coef[i])

In [ ]:
from sklearn.feature_selection import RFECV
rfe = RFECV(estimator = LogisticRegression(random_state=random_state), cv=10)
# run the RFECV
rfe.fit(X_train, y_train)
# comparing how many variables before and after
print("Original feature set", X_train.shape[1])
print("Number of features after elimination", rfe.n_features_)

In [ ]:
X_train_sel = rfe.transform(X_train)
X_test_sel = rfe.transform(X_test)

In [ ]:
# grid search CV
params = {
    'C': np.logspace(-3, 3, 9),
    'solver': ['lbfgs', 'newton-cg'],
    'class_weight': [None, 'balanced'],
    'max_iter': [1000],
    'tol': [1e-4, 1e-3, 1e-2]
}


# use all cores to tune logistic regression with C parameter
cvsel = GridSearchCV(param_grid=params,
estimator=LogisticRegression(random_state=random_state), cv=10, n_jobs=-1, scoring='f1')
# swapped to recall scoring - it was predicing
cvsel.fit(X_train_sel, y_train)

# test the best model
print("Train accuracy:", cvsel.score(X_train_sel, y_train))
print("Test accuracy:", cvsel.score(X_test_sel, y_test))

y_pred = cvsel.predict(X_test_sel)
print(classification_report(y_test, y_pred))

# print parameters of the best model
print(cvsel.best_params_)

In [ ]:
best_regression_rfe_model = cvsel.best_estimator_
regression_X_train = X_train_sel
regression_y_train = y_train
regression_X_test = X_test_sel
regression_y_test = y_test

In [ ]:
# grab feature importances from the model and feature name from the original X
modelsel = cvsel.best_estimator_
coef = modelsel.coef_[0]
feature_names = X.columns
# sort them out in descending order
indices = np.argsort(np.absolute(coef))[::-1]
#indices = np.flip(indices, axis=0)

for i in indices:
  print(feature_names[i], ':', coef[i])

In [ ]:
y_pred_proba_cv = cv.predict_proba(X_test)
y_pred_proba_cvsel = cvsel.predict_proba(X_test_sel)
roc_index_cv = roc_auc_score(y_test, y_pred_proba_cv[:, 1])
roc_index_cvsel = roc_auc_score(y_test, y_pred_proba_cvsel[:, 1])
print("ROC index on test for `cv`:", roc_index_cv)
print("ROC index on test for `cv sel`:", roc_index_cvsel)


In [ ]:
fpr_lr, tpr_lr, thresholds_lr = roc_curve(y_test, y_pred_proba_cv[:,1])
fpr_lr_cv, tpr_lr_cv, thresholds_lr_cv = roc_curve(y_test, y_pred_proba_cvsel[:,1])

plt.plot(fpr_lr, tpr_lr, label=f'ROC Curve for `cv` {roc_index_cv:.3f}', color='red',
lw=0.5)
plt.plot(fpr_lr_cv, tpr_lr_cv, label=f'ROC Curve for `cv RFECV` {roc_index_cvsel:.3f}',
color='green', lw=0.5)

plt.plot([0, 1], [0, 1], color='navy', lw=0.5, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()


# Neural Network Model

In [ ]:
dfnn = dfregression.copy()
# Obtain a list of labels (targets) for each of the samples. We refer to this as 'y'.
Y = dfnn["IsBadBuy"]
# Remove the label (target) from the samples, as this is not a feature that we want to use to predict against.
X = dfnn.drop(['IsBadBuy'], axis=1)
# Create a 30% train/test split from our samples.
# We set the random_state to the constant 10 to ensure it is the same from run-to-run.
# We set the stratisfy parameter to the labels, that way we get an even selection of 0 and 1 values in our testing set.
# Otherwise, we may end up with unbalanced 0/1 labels across the two splits.
# We use X.values to get arrays for the features, instead of a DataFrame with column names etc.
random_state = 10
test_set_size = 0.3 # 30%
X_train, X_test, y_train, y_test = train_test_split(X.values, Y, test_size=test_set_size,
stratify=Y, random_state=random_state)
print("Size of training set:", len(X_train))
print("Size of testing set:", len(X_test))

In [ ]:
# initialise a standard scaler object
scaler = StandardScaler()

# visualise min, max, mean and standard dev of data before scaling
print("Before scaling\n-------------")
for i in range(5):
  col = X_train[:,i]
  print("Variable #{}: min {}, max {}, mean {:.2f} and std dev {:.2f}".format(i,
min(col), max(col), np.mean(col), np.std(col)))

# learn the mean and std.dev of variables from training data
# then use the learned values to transform training data
X_train = scaler.fit_transform(X_train, y_train)
print("After scaling\n-------------")
for i in range(5):
  col = X_train[:,i]
  print("Variable #{}: min {}, max {}, mean {:.2f} and std dev {:.2f}".format(i,
min(col), max(col), np.mean(col), np.std(col)))

# use the statistic that you learned from training to transform test data
# NEVER learn from test data, this is supposed to be a set of dataset
# that the model has never seen before
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
model = MLPClassifier(random_state=random_state)
model.fit(X_train, y_train)

In [ ]:
print("Train accuracy:", model.score(X_train, y_train))
print("Test accuracy:", model.score(X_test, y_test))
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
params = {
    'hidden_layer_sizes': [(50,), (50,20), (100)],
    'alpha': [0.001, 0.0001, 0.00001],
    'learning_rate_init': [0.001, 0.0001, 0.00001],
    'max_iter': [1500],
    }

cv_1 = GridSearchCV(param_grid=params,
estimator=MLPClassifier(random_state=random_state), return_train_score=True, cv=10, n_jobs=-1, scoring='f1')
cv_1.fit(X_train, y_train)

In [ ]:
print("Train accuracy:", cv_1.score(X_train, y_train))
print("Test accuracy:", cv_1.score(X_test, y_test))
y_pred = cv_1.predict(X_test)
print(classification_report(y_test, y_pred))
print(cv_1.best_params_)

In [ ]:
from sklearn.feature_selection import SelectFromModel

selectmodel = SelectFromModel(model_grid, prefit=False)
X_train_sel_model = selectmodel.transform(X_train)
X_test_sel_model = selectmodel.transform(X_test)
print(X_train_sel_model.shape)


mask = selectmodel.get_support()
selected_features = feature_names[mask]

print("Selected features:")
print(selected_features)

In [ ]:
params = {
    'hidden_layer_sizes': [(50,), (50,20), (100)],
    'alpha': [0.001, 0.0001, 0.00001],
    'learning_rate_init': [0.001, 0.0001, 0.00001],
    'max_iter': [1500],
    }
cv_sel_model = GridSearchCV(param_grid=params, estimator=MLPClassifier(random_state=random_state), cv=10, n_jobs=-1, scoring='f1')
cv_sel_model.fit(X_train_sel_model, y_train)

y_pred = cv_sel_model.predict(X_test_sel_model)


In [ ]:
print("Train accuracy:", cv_sel_model.score(X_train_sel_model, y_train))
print("Test accuracy:", cv_sel_model.score(X_test_sel_model, y_test))
y_pred = cv_sel_model.predict(X_test_sel_model)
print(classification_report(y_test, y_pred))
print(cv_sel_model.best_params_)

In [ ]:
print("Basic CV NN Model")
print(cv_1.best_estimator_.n_iter_)
print("Feature Reduced CV NN Model")
print(cv_sel_model.best_estimator_.n_iter_)


In [ ]:
y_pred_proba_cv_1 = cv_1.predict_proba(X_test)
y_pred_proba_cv_sel_model = cv_sel_model.predict_proba(X_test_sel_model)
roc_index_cv_1 = roc_auc_score(y_test, y_pred_proba_cv_1[:, 1])
roc_index_cv_sel_model = roc_auc_score(y_test, y_pred_proba_cv_sel_model[:, 1])
print("ROC index on test for NN with gridsearch 1:", roc_index_cv_1)
print("ROC index on test for NN with feature selection (model selection) and gridsearch:", roc_index_cv_sel_model)

fpr_cv_1, tpr_cv_1, thresholds_cv_1 = roc_curve(y_test, y_pred_proba_cv_1[:,1])
fpr_cv_sel_model, tpr_cv_sel_model, thresholds_cv_sel_model = roc_curve(y_test, y_pred_proba_cv_sel_model[:,1])

plt.plot(fpr_cv_1, tpr_cv_1, label='Basic CV NN Model {:.3f}'.format(roc_index_cv_1), color='blue',
lw=0.5)
plt.plot(fpr_cv_sel_model, tpr_cv_sel_model, label='Feature Reduced CV NN Model {:.3f}'.format(roc_index_cv_sel_model), color='red', lw=0.5)
plt.plot([0, 1], [0, 1], color='navy', lw=0.5, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
for i, coef in enumerate(cv_sel_model.best_estimator_.coefs_):
    print(f"Layer {i} weights shape: {coef.shape}")
    print(coef)

In [ ]:
y_pred_proba_tree = best_tree_model.predict_proba(tree_X_test)
y_pred_proba_regression = best_regression_rfe_model.predict_proba(regression_X_test)
y_pred_proba_nn = cv_sel_model.predict_proba(X_test_sel_model)

roc_index_tree = roc_auc_score(tree_y_test, y_pred_proba_tree[:, 1])
roc_index_regression = roc_auc_score(regression_y_test, y_pred_proba_regression[:, 1])
roc_index_cv_sel_model = roc_auc_score(y_test, y_pred_proba_cv_sel_model[:, 1])

print("ROC index on test for Best Tree:", roc_index_tree)
print("ROC index on test for Best Regression:", roc_index_regression)
print("ROC index on test for Best NN:", roc_index_cv_sel_model)

fpr_tree, tpr_tree, thresholds_tree = roc_curve(tree_y_test, y_pred_proba_tree[:,1])
fpr_regression, tpr_regression, thresholds_regression = roc_curve(regression_y_test, y_pred_proba_regression[:,1])
fpr_cv_sel_model, tpr_cv_sel_model, thresholds_cv_sel_model = roc_curve(y_test, y_pred_proba_cv_sel_model[:,1])

plt.plot(fpr_tree, tpr_tree, label='Best Tree Model {:.3f}'.format(roc_index_tree), color='Green',
lw=0.5)
plt.plot(fpr_regression, tpr_regression, label='Best Regression Model {:.3f}'.format(roc_index_regression), color='blue',
lw=0.5)
plt.plot(fpr_cv_sel_model, tpr_cv_sel_model, label='Best NN Model {:.3f}'.format(roc_index_cv_sel_model), color='red', lw=0.5)

plt.plot([0, 1], [0, 1], color='navy', lw=0.5, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()